# OpenStreetMap (OSM) data for Podgorica, Montenegro

In [ ]:
# Loading of packages required for this Notebook

%pip install quackosm folium matplotlib mapclassify ipywidgets --quiet

### Finding OSM data for area of Podgorica (Montenegro)

In [ ]:
import quackosm as qosm

# Finding geometry of Podgorica
area_podgorica = qosm.geocode_to_geometry("Capital Podgorica, Montenegro")
area_podgorica

In [ ]:
# Find suitable PBF file(s) of OSM that covers Podgorica geometry 
# And filter out OSM features only for that area
gdf = qosm.convert_geometry_to_geodataframe(area_podgorica)
gdf

### Explore extracted OSM features

In [ ]:

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import geopandas as gpd

fig = plt.figure(figsize=(10, 10))
ax = fig.subplots()

gdf.plot(ax=ax, markersize=1, zorder=1, alpha=0.4)
gdf.boundary.plot(ax=ax, markersize=0, zorder=1, alpha=0.8)
gpd.GeoSeries([area_podgorica], crs=4326).plot(
    ax=ax,
    color=(0, 0, 0, 0),
    zorder=2,
    hatch="///",
    edgecolor="orange",
    linewidth=1.5,
    
)

blue_patch = mpatches.Patch(color="C0", alpha=0.8, label="Extracted OSM features")
orange_patch = mpatches.Patch(
    facecolor=(0, 0, 0, 0), edgecolor="orange", hatch="///", linewidth=1.5, label="Podgorica Geometry filter"
)
ax.legend(handles=[blue_patch, orange_patch], loc="lower right")
plt.show()

### Explore facilities of education in Podgorica

In [24]:
import geopandas as gpd

filter_vrtic ={"building": "kindergarten", "amenity": "kindergarten"}
filter_school ={"building": "school", "amenity": "school"}
filter_university ={"building": "university", "amenity": "university"}


gdf = qosm.convert_geometry_to_geodataframe(area_podgorica, tags_filter=filter_vrtic, verbosity_mode="transient", keep_all_tags=True, explode_tags=True)
gdf1 = qosm.convert_geometry_to_geodataframe(area_podgorica, tags_filter=filter_school, verbosity_mode="transient", keep_all_tags=True, explode_tags=True)
gdf2 = qosm.convert_geometry_to_geodataframe(area_podgorica, tags_filter=filter_university, verbosity_mode="transient", keep_all_tags=True, explode_tags=True)

m = gdf.explore(color="orangered", tiles="CartoDB positron")
m2 = gdf1.explore(color="blue", m=m)
m3 = gdf2.explore(color="pink", m=m)


gpd.GeoSeries([area_podgorica], crs=4326).boundary.explore(m=m)

Exploring public transportation data in OSM

In [32]:
import duckdb
import geopandas
# Load bus routes data using DuckDB
duckdb.load_extension("spatial")
bus_routes_unnested = duckdb.sql(
    """
    SELECT
        id,
        tags as route_tags,
        UNNEST(refs) as ref,
        UNNEST(ref_types) as ref_type,
        UNNEST(ref_roles) as ref_role
    FROM ST_ReadOSM("files/geofabrik_europe_montenegro.osm.pbf")
    WHERE kind = 'relation'
    AND tags['route'][1] = 'bus'
    """
).to_df()
# Join the data
bus_routes_unnested["feature_id"] = (
    bus_routes_unnested["ref_type"].astype(str) + "/" + bus_routes_unnested["ref"].astype(str)
)

# Load highway and public_transport features
highway_filter = {"highway": True, "public_transport": True}

features_gdf = qosm.convert_osm_extract_to_geodataframe(
    "geofabrik_europe_montenegro", tags_filter=highway_filter, keep_all_tags=True, geometry_filter=area_podgorica
)

features_gdf.explore()

bus_routes_with_geometries = bus_routes_unnested.merge(features_gdf, left_on="feature_id", right_index=True)
bus_routes_with_geometries = bus_routes_with_geometries.rename(
    columns={
        "id": "relation_route_id",
        "tags": "feature_tags",
    }
)

gdf = geopandas.GeoDataFrame(bus_routes_with_geometries, geometry="geometry", crs=4326)
gpd.GeoSeries([area_podgorica], crs=4326).boundary.explore(m=gdf.explore(color="orangered", tiles="CartoDB positron"))

/home/panda/projects/.venv/lib/python3.10/site-packages/geopandas/array.py:1638: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))
